In [2]:
import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup

In [12]:
!conda install -c conda-forge geopy --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [13]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         673 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.1-py_0 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


vincent-0.4.4        | 28 KB     | ##################################### | 1

In [20]:
!pip install geocoder

     |████████████████████████████████| 102kB 16.9MB/s ta 0:00:01


# Data Scraping from Wikepedia for Cairo and Giza in Egypt 

In [372]:
url = "https://en.wikipedia.org/wiki/Cairo_Governorate"
req = urllib.request.urlopen(url)
article = req.read().decode()

soup = BeautifulSoup(article, 'html.parser')
table = soup.find('table', class_='sortable')

In [373]:
url2 = "https://en.wikipedia.org/wiki/Giza_Governorate#Overview"
req2 = urllib.request.urlopen(url2)
article2 = req2.read().decode()

soup2 = BeautifulSoup(article2, 'html.parser')
table2 = soup2.find('table', class_='sortable')

In [374]:
ths = table.find_all('th')
headings = [th.text.strip() for th in ths]

#create dataframe
neighbour = pd.DataFrame(columns=headings)
neighbour

# scrape rows into dataframe
for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue                            #skips first row with headings
    An, b, n,pop,ty = [td.text.strip() for td in tds]
    if b!="Not assigned":
        neighbour = neighbour.append({'Anglicized name': An, 'Native name': b, 'Arabic transliteration': n,'Population(July 2017 Est.)':pop,'Type':ty}, ignore_index=True)

neighbour.head()

,Anglicized name,Native name,Arabic transliteration,Population(July 2017 Est.),Type
0,15 May City,قسم 15 مايو,Māyū,"93,879",Kism (fully urban)
1,Abdeen,قسم عابدين,'Ābidīn,"40,450",Kism (fully urban)
2,El Darb El Ahmar,قسم الدرب الأحمر,Al-Darb Al-Aḥmar[10],"58,677",Kism (fully urban)
3,Ain Shams,قسم عين شمس,'Ain Schams,"616,374",Kism (fully urban)
4,Amreya,قسم الاميريه,Al-Amīriīah,"153,046",Kism (fully urban)


In [375]:
ths2 = table2.find_all('th')
headings2 = [th.text.strip() for th in ths2]

#create dataframe
neighbour2 = pd.DataFrame(columns=headings2)
neighbour2

# scrape rows into dataframe
for tr in table2.find_all('tr'):
    tds2 = tr.find_all('td')
    if not tds2:
        continue                            #skips first row with headings
    An, b, n,pop,ty = [td.text.strip() for td in tds2]
    if b!="Not assigned":
        neighbour2 = neighbour2.append({'Anglicized name': An, 'Native name': b, 'Arabic transliteration': n,'Population(July 2017 Est.)':pop,'Type':ty}, ignore_index=True)

neighbour2.head()

,Anglicized name,Native name,Arabic transliteration,Population(July 2017 Est.),Type
0,Dokki,قسم الدقي,Ad-Duqqī,"71,161",Kism (fully urban)
1,Pyramids,قسم الأهرام,Al-Ahrām,"661,478",Kism (fully urban)
2,Agouza,قسم العجوزة,Al-'Ajūzah,"279,400",Kism (fully urban)
3,El Ayyat,مركز العياط,Al-'Ayyāṭ,"494,548",Markaz
4,El Badrashein,مركز البدرشين,Al-Badrashayn,"539,656",Markaz


# Getting the Coordinates for the scrapped Data

In [376]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Cairo, Egypt'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [377]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

In [378]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in neighbour["Anglicized name"].tolist() ]

In [379]:
# call the function to get the coordinates, store in a new list using list comprehension
coords2 = [ get_latlng(neighborhood) for neighborhood in neighbour2["Anglicized name"].tolist() ]

In [136]:
coords

[[29.829830000000072, 31.369650000000036],
 [30.045420000000036, 31.24210000000005],
 [29.977690000000052, 31.26361000000003],
 [30.135420000000067, 31.323890000000063],
 [30.11238000000003, 31.291330000000073],
 [30.049920000000043, 31.248600000000067],
 [29.89363000000003, 31.295180000000073],
 [30.09895000000006, 31.28944000000007],
 [29.964960000000076, 31.251520000000028],
 [29.98416000000003, 31.271630000000073],
 [30.156510000000026, 31.34459000000004],
 [29.90441000000004, 31.29884000000004],
 [30.093450000000075, 31.29299000000003],
 [30.01293000000004, 31.33809000000008],
 [30.048748509002316, 31.25651102964668],
 [30.082929820705175, 31.261068555369636],
 [30.16028619599679, 31.40418245379753],
 [30.08284882070518, 31.261050555369625],
 [30.07206000000008, 31.28349000000003],
 [30.111000000000047, 31.34749000000005],
 [30.089260000000024, 31.236870000000067],
 [30.070050000000037, 31.259640000000047],
 [30.106880000000046, 31.288040000000024],
 [30.049920000000043, 31.248600

In [128]:
coords2

[[30.050070000000062, 31.27192000000008],
 [29.84698000000003, 31.332810000000052],
 [29.893120000000067, 31.30319000000003],
 [30.040050000000065, 31.255300000000034],
 [30.03672000000006, 31.241910000000075],
 [29.900000000000034, 31.250000000000057],
 [29.989460000000065, 31.216890000000035],
 [29.989460000000065, 31.216890000000035],
 [29.990180000000066, 31.190570000000037],
 [28.938504875660126, 29.471383210989497],
 [30.138270000000034, 31.38671000000005],
 [30.047330000000045, 30.986790000000042],
 [30.112211390105898, 31.61458846428745],
 [30.049920000000043, 31.248600000000067],
 [30.049920000000043, 31.248600000000067],
 [30.049920000000043, 31.248600000000067],
 [30.049920000000043, 31.248600000000067],
 [30.07354500000001, 31.230049500000007],
 [30.07354500000001, 31.230049500000007],
 [29.940770000000043, 31.27696000000003],
 [29.854000000000042, 31.33454000000006],
 [30.05284566518158, 31.398814130454674]]

In [380]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [381]:
df_coords2 = pd.DataFrame(coords2, columns=['Latitude', 'Longitude'])

In [382]:
# merge the coordinates into the original dataframe
neighbour['Latitude'] = df_coords['Latitude']
neighbour['Longitude'] = df_coords['Longitude']

In [383]:
neighbour2['Latitude'] = df_coords2['Latitude']
neighbour2['Longitude'] = df_coords2['Longitude']

In [384]:
print(neighbour2.shape)
neighbour2

(22, 7)


,Anglicized name,Native name,Arabic transliteration,Population(July 2017 Est.),Type,Latitude,Longitude
0,Dokki,قسم الدقي,Ad-Duqqī,"71,161",Kism (fully urban),30.050070,31.271920
1,Pyramids,قسم الأهرام,Al-Ahrām,"661,478",Kism (fully urban),29.846980,31.332810
2,Agouza,قسم العجوزة,Al-'Ajūzah,"279,400",Kism (fully urban),29.893120,31.303190
3,El Ayyat,مركز العياط,Al-'Ayyāṭ,"494,548",Markaz,30.040050,31.255300
4,El Badrashein,مركز البدرشين,Al-Badrashayn,"539,656",Markaz,30.036720,31.241910
5,El Hawamdeya,قسم الحوامدية,Al-Ḥawāmidiyah,"191,552",Kism (urban and rural parts),29.900000,31.250000
6,Giza,قسم الجيزة,Al-Jīzah,"286,668",Kism (fully urban),29.989460,31.216890
7,Giza,مركز الجيزة,Al-Jīzah,"369,351",Markaz,29.989460,31.216890
8,El Omraniya,قسم العمرانية,Al-'Umrāniyah,"367,274",Kism (fully urban),29.990180,31.190570
9,El Wahat El Bahariya,قسم الواحات البحرية,Al-Wāḥāt al-Baḥariyah,"39,415",Kism (urban and rural parts),28.938505,29.471383


In [385]:
neighbour=neighbour.append(neighbour2)

In [386]:
neighbour=neighbour.reset_index()

In [387]:
neighbour

,index,Anglicized name,Native name,Arabic transliteration,Population(July 2017 Est.),Type,Latitude,Longitude
0,0,15 May City,قسم 15 مايو,Māyū,"93,879",Kism (fully urban),29.829830,31.369650
1,1,Abdeen,قسم عابدين,'Ābidīn,"40,450",Kism (fully urban),30.045420,31.242100
2,2,El Darb El Ahmar,قسم الدرب الأحمر,Al-Darb Al-Aḥmar[10],"58,677",Kism (fully urban),29.977690,31.263610
3,3,Ain Shams,قسم عين شمس,'Ain Schams,"616,374",Kism (fully urban),30.135420,31.323890
4,4,Amreya,قسم الاميريه,Al-Amīriīah,"153,046",Kism (fully urban),30.112380,31.291330
5,5,Azbakeya,قسم الأزبكية,Al-Azbakiyah,"19,826",Kism (fully urban),30.049920,31.248600
6,6,El Basatin,قسم البساتين,Al-Basātīn,"497,041",Kism (fully urban),29.893630,31.295180
7,7,El Gamaliya,قسم الجمالية,Al-Jamāliyah,"36,485",Kism (fully urban),30.098950,31.289440
8,8,El Khalifa,قسم الخليفة,Al-Khalīfah,"105,578",Kism (fully urban),29.964960,31.251520
9,9,Maadi,قسم المعادي,Al-Ma'ādī,"88,869",Kism (fully urban),29.984160,31.271630


In [388]:
neighbour=neighbour.drop(['Native name', 'Arabic transliteration','Type'], axis=1)

In [389]:
neighbour=neighbour.rename(columns={"Anglicized name": "region"})

In [390]:
neighbour=neighbour.rename(columns={"Population(July 2017 Est.)": "Population"})

In [391]:
neighbour

,index,region,Population,Latitude,Longitude
0,0,15 May City,"93,879",29.829830,31.369650
1,1,Abdeen,"40,450",30.045420,31.242100
2,2,El Darb El Ahmar,"58,677",29.977690,31.263610
3,3,Ain Shams,"616,374",30.135420,31.323890
4,4,Amreya,"153,046",30.112380,31.291330
5,5,Azbakeya,"19,826",30.049920,31.248600
6,6,El Basatin,"497,041",29.893630,31.295180
7,7,El Gamaliya,"36,485",30.098950,31.289440
8,8,El Khalifa,"105,578",29.964960,31.251520
9,9,Maadi,"88,869",29.984160,31.271630


In [395]:
neighbour= neighbour.drop([61], axis=0)

In [393]:
neighbour=neighbour.drop(['index'], axis=1)

In [394]:
neighbour.columns

Index(['region', 'Population', 'Latitude', 'Longitude'], dtype='object')

In [209]:
neighbour=neighbour.drop(['level_0'], axis=1)

In [396]:
neighbour

,region,Population,Latitude,Longitude
0,15 May City,"93,879",29.829830,31.369650
1,Abdeen,"40,450",30.045420,31.242100
2,El Darb El Ahmar,"58,677",29.977690,31.263610
3,Ain Shams,"616,374",30.135420,31.323890
4,Amreya,"153,046",30.112380,31.291330
5,Azbakeya,"19,826",30.049920,31.248600
6,El Basatin,"497,041",29.893630,31.295180
7,El Gamaliya,"36,485",30.098950,31.289440
8,El Khalifa,"105,578",29.964960,31.251520
9,Maadi,"88,869",29.984160,31.271630


In [397]:
address = 'Cairo, Egypt'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Cairo, Egypt {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Cairo, Egypt 30.048819, 31.243666.


In [398]:
import folium
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(neighbour['Latitude'],neighbour['Longitude'],neighbour['region']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [169]:
CLIENT_ID = 'A5WKVOBBJYLV5NXMFJUJPJHQQKPKKOBOCHTGLWR5BFWDGSML' # your Foursquare ID
CLIENT_SECRET = 'C3L5UD3MHWETT0EDNRFZQJQLBPEPTEBUUBYQVKKDG5ERUKKZ' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A5WKVOBBJYLV5NXMFJUJPJHQQKPKKOBOCHTGLWR5BFWDGSML
CLIENT_SECRET:C3L5UD3MHWETT0EDNRFZQJQLBPEPTEBUUBYQVKKDG5ERUKKZ


In [399]:
import requests # library to handle requests
radius = 1000
LIMIT = 400

venues = []

for lat, long, neighborhood in zip(neighbour['Latitude'], neighbour['Longitude'], neighbour['region']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [400]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['region', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1738, 7)


,region,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,15 May City,29.82983,31.36965,Downtown 15 May,29.827649,31.364928,Coffee Shop
1,15 May City,29.82983,31.36965,Pochinki,29.827959,31.369610,Coffee Shop
2,15 May City,29.82983,31.36965,On The Roof Café,29.824021,31.366160,Café
3,15 May City,29.82983,31.36965,Dr. Ahmed AbdelSalam Pharamcy,29.823380,31.366846,Pharmacy
4,15 May City,29.82983,31.36965,15 May Microbus Stop,29.832846,31.378454,Bus Station


In [337]:
coc=pd.DataFrame(venues_df.loc[venues_df['VenueCategory']=='Hotel'])

In [338]:
coc

,region,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
44,Abdeen,30.045420,31.242100,Steigenberger El Tahrir Hotel,30.046732,31.235610,Hotel
49,Abdeen,30.045420,31.242100,Windsor Hotel Cairo,30.053602,31.245593,Hotel
130,Azbakeya,30.049920,31.248600,Windsor Hotel Cairo,30.053602,31.245593,Hotel
237,El Khalifa,29.964960,31.251520,Holiday Inn Cairo Maadi,29.960753,31.247909,Hotel
359,El Weili,30.072060,31.283490,Armored Vehicles House (دار المركبات),30.064848,31.288221,Hotel
410,El Nozha,30.111000,31.347490,Heliopark Hotel,30.109443,31.340791,Hotel
446,El Segil,30.049920,31.248600,Windsor Hotel Cairo,30.053602,31.245593,Hotel
577,El Sayeda Zeinab,30.032409,31.239037,Nile Seasons Hotel,30.034900,31.229361,Hotel
587,El Zaher,30.062530,31.254790,Ciao Hotel (فندق تشاو),30.065279,31.247622,Hotel
593,Zamalek,30.055860,31.223900,Marriott Gardens,30.057039,31.224651,Hotel


In [432]:
venues_df.VenueCategory.unique()

164

In [402]:
venues_df.groupby(["region"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
region,,,,,,
15 May City,7,7,7,7,7,7
6 October City (1 & 3),8,8,8,8,8,8
6th of October City (2),10,10,10,10,10,10
Abdeen,100,100,100,100,100,100
Agouza,4,4,4,4,4,4
Ain Shams,9,9,9,9,9,9
Amreya,4,4,4,4,4,4
Atfeh,81,81,81,81,81,81
Azbakeya,81,81,81,81,81,81


In [533]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['region'] = venues_df['region'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(1738, 165)


,region,Accessories Store,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Track,Tunnel,Turkish Restaurant,Used Bookstore,Waterfront,Yoga Studio
0,15 May City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15 May City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15 May City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15 May City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15 May City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [534]:
kl_grouped = kl_onehot.groupby(["region"]).sum().reset_index()

print(kl_grouped.shape)
kl_grouped

(61, 165)


,region,Accessories Store,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Track,Tunnel,Turkish Restaurant,Used Bookstore,Waterfront,Yoga Studio
0,15 May City,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6 October City (1 & 3),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6th of October City (2),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abdeen,0,0,0,3,0,1,0,0,0,...,0,0,4,0,0,1,0,0,0,0
4,Agouza,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Ain Shams,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Amreya,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Atfeh,1,0,0,1,1,0,0,0,0,...,0,0,5,0,0,1,0,1,0,0
8,Azbakeya,1,0,0,1,1,0,0,0,0,...,0,0,5,0,0,1,0,1,0,0
9,Bab El Sharia,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [581]:
kl_grouped.columns

Index(['region', 'Accessories Store', 'American Restaurant', 'Aquarium',
       'Art Gallery', 'Art Museum', 'Arts & Crafts Store',
       'Arts & Entertainment', 'Asian Restaurant', 'Athletics & Sports',
       ...
       'Tex-Mex Restaurant', 'Thai Restaurant', 'Theater',
       'Theme Park Ride / Attraction', 'Track', 'Tunnel', 'Turkish Restaurant',
       'Used Bookstore', 'Waterfront', 'Yoga Studio'],
      dtype='object', length=165)

In [530]:
kl_grouped.loc[kl_grouped['region']=='Qasr El Nil']

,region,Accessories Store,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,...,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Track,Tunnel,Turkish Restaurant,Used Bookstore,Waterfront,Yoga Studio
54,Qasr El Nil,0,0,0,2,0,1,0,1,0,...,1,1,2,0,0,0,1,0,1,0


In [536]:
kl_grouped2 = kl_grouped.loc[:,"Accessories Store":"Yoga Studio"].div(kl_grouped.sum(axis=0))

In [537]:
kl_grouped2

,Accessories Store,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,...,Thai Restaurant,Theater,Theme Park Ride / Attraction,Track,Tunnel,Turkish Restaurant,Used Bookstore,Waterfront,Yoga Studio,region
0,0.000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000,0.00,0.000000,NaN
1,0.000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000,0.00,0.000000,NaN
2,0.000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000,0.00,0.000000,NaN
3,0.000,0.0,0.0,0.166667,0.0,0.333333,0.000000,0.000000,0.0,0.0,...,0.0,0.085106,0.0,0.00,0.076923,0.000000,0.000,0.00,0.000000,NaN
4,0.000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000,0.00,0.000000,NaN
5,0.000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000,0.00,0.000000,NaN
6,0.000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000,0.00,0.000000,NaN
7,0.125,0.0,0.0,0.055556,0.1,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.106383,0.0,0.00,0.076923,0.000000,0.125,0.00,0.000000,NaN
8,0.125,0.0,0.0,0.055556,0.1,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.106383,0.0,0.00,0.076923,0.000000,0.125,0.00,0.000000,NaN
9,0.125,0.0,0.0,0.000000,0.0,0.000000,0.333333,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000,0.00,0.000000,NaN


In [538]:
kl_grouped2['region']=kl_grouped['region']

In [540]:
cols = list(kl_grouped2.columns)
cols = [cols[-1]] + cols[:-1]
kl_grouped2 = kl_grouped2[cols]

In [548]:
kl_grouped = kl_grouped2

In [582]:
len(kl_grouped[kl_grouped["Coffee Shop"] > 0])

25

In [583]:
kl_cof = kl_grouped[["region","Coffee Shop"]]
kl_cof.head()

,region,Coffee Shop
0,15 May City,0.028571
1,6 October City (1 & 3),0.000000
2,6th of October City (2),0.000000
3,Abdeen,0.057143
4,Agouza,0.014286


In [584]:
# set number of clusters
kclusters = 3
kl_clustering = kl_cof.drop(["region"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 1, 2, 1, 1, 1, 2, 2, 1], dtype=int32)

In [585]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_cof.copy()
# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_
kl_merged.head()

,region,Coffee Shop,Cluster Labels
0,15 May City,0.028571,2
1,6 October City (1 & 3),0.000000,1
2,6th of October City (2),0.000000,1
3,Abdeen,0.057143,2
4,Agouza,0.014286,1


In [586]:

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(neighbour.set_index("region"), on="region")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(64, 6)


,region,Coffee Shop,Cluster Labels,Population,Latitude,Longitude
0,15 May City,0.028571,2,"93,879",29.829830,31.369650
1,6 October City (1 & 3),0.000000,1,"152,999",30.052846,31.398814
2,6th of October City (2),0.000000,1,"197,019",29.854000,31.334540
3,Abdeen,0.057143,2,"40,450",30.045420,31.242100
4,Agouza,0.014286,1,"279,400",29.893120,31.303190


In [587]:

# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(64, 6)


,region,Coffee Shop,Cluster Labels,Population,Latitude,Longitude
59,Zamalek,0.100000,0,"14,993",30.055860,31.223900
11,Bulaq,0.114286,0,"48,301",30.049920,31.248600
11,Bulaq,0.114286,0,"963,201",30.049920,31.248600
29,El Salam,0.000000,1,"154,267",30.157800,31.402070
30,El Sayeda Zeinab,0.014286,1,"136,722",30.032409,31.239037
32,El Sharabiya,0.000000,1,"187,806",30.070050,31.259640
33,El Shorouk,0.000000,1,"87,565",30.106880,31.288040
34,El Warraq,0.000000,1,"724,963",30.138270,31.386710
35,El Weili,0.014286,1,"79,548",30.072060,31.283490
36,El Zaher,0.000000,1,"72,101",30.062530,31.254790


In [405]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [433]:
num_top_venues = 164

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['region']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['region'] = kl_grouped['region']

for ind in np.arange(kl_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(kl_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,155th Most Common Venue,156th Most Common Venue,157th Most Common Venue,158th Most Common Venue,159th Most Common Venue,160th Most Common Venue,161th Most Common Venue,162th Most Common Venue,163th Most Common Venue,164th Most Common Venue
0,15 May City,Coffee Shop,Café,Market,Bus Station,Pharmacy,Egyptian Restaurant,Fast Food Restaurant,Food Stand,Distribution Center,...,Nightclub,Movie Theater,New American Restaurant,Neighborhood,Music Venue,Music School,Museum,Multiplex,Moving Target,Accessories Store
1,6 October City (1 & 3),Dessert Shop,Shopping Mall,Bus Station,Sports Club,Café,Resort,Falafel Restaurant,Juice Bar,Egyptian Restaurant,...,New American Restaurant,Motorcycle Shop,Neighborhood,Music Venue,Music School,Museum,Multiplex,Moving Target,Movie Theater,Accessories Store
2,6th of October City (2),Supermarket,Café,Park,Outdoor Supply Store,Fried Chicken Joint,Seafood Restaurant,Gym,Farm,Food & Drink Shop,...,Neighborhood,Mosque,Music Venue,Music School,Museum,Multiplex,Moving Target,Movie Theater,Motorcycle Shop,Accessories Store
3,Abdeen,Café,Egyptian Restaurant,Plaza,Theater,Coffee Shop,Dessert Shop,Art Gallery,Middle Eastern Restaurant,Pastry Shop,...,Movie Theater,Moving Target,Multiplex,Museum,Music School,New American Restaurant,Opera House,Optical Shop,Organic Grocery,Accessories Store
4,Agouza,Coffee Shop,Mobile Phone Shop,Café,Fast Food Restaurant,Farm,Food Stand,Food Court,Food & Drink Shop,Flea Market,...,Opera House,Moving Target,Nightclub,New American Restaurant,Neighborhood,Music Venue,Music School,Museum,Multiplex,Accessories Store


In [438]:
neighborhoods_venues_sorted

,Cluster Labels,region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,155th Most Common Venue,156th Most Common Venue,157th Most Common Venue,158th Most Common Venue,159th Most Common Venue,160th Most Common Venue,161th Most Common Venue,162th Most Common Venue,163th Most Common Venue,164th Most Common Venue
0,0,15 May City,Coffee Shop,Café,Market,Bus Station,Pharmacy,Egyptian Restaurant,Fast Food Restaurant,Food Stand,...,Nightclub,Movie Theater,New American Restaurant,Neighborhood,Music Venue,Music School,Museum,Multiplex,Moving Target,Accessories Store
1,0,6 October City (1 & 3),Dessert Shop,Shopping Mall,Bus Station,Sports Club,Café,Resort,Falafel Restaurant,Juice Bar,...,New American Restaurant,Motorcycle Shop,Neighborhood,Music Venue,Music School,Museum,Multiplex,Moving Target,Movie Theater,Accessories Store
2,0,6th of October City (2),Supermarket,Café,Park,Outdoor Supply Store,Fried Chicken Joint,Seafood Restaurant,Gym,Farm,...,Neighborhood,Mosque,Music Venue,Music School,Museum,Multiplex,Moving Target,Movie Theater,Motorcycle Shop,Accessories Store
3,0,Abdeen,Café,Egyptian Restaurant,Plaza,Theater,Coffee Shop,Dessert Shop,Art Gallery,Middle Eastern Restaurant,...,Movie Theater,Moving Target,Multiplex,Museum,Music School,New American Restaurant,Opera House,Optical Shop,Organic Grocery,Accessories Store
4,0,Agouza,Coffee Shop,Mobile Phone Shop,Café,Fast Food Restaurant,Farm,Food Stand,Food Court,Food & Drink Shop,...,Opera House,Moving Target,Nightclub,New American Restaurant,Neighborhood,Music Venue,Music School,Museum,Multiplex,Accessories Store
5,0,Ain Shams,Gym,Seafood Restaurant,Egyptian Restaurant,Electronics Store,Café,Mobile Phone Shop,Kitchen Supply Store,Metro Station,...,Opera House,Moving Target,Nightclub,New American Restaurant,Neighborhood,Music Venue,Music School,Museum,Multiplex,Accessories Store
6,3,Amreya,Bakery,Kebab Restaurant,Egyptian Restaurant,Café,Yoga Studio,Food Truck,Food Stand,Food Court,...,Opera House,Moving Target,Nightclub,New American Restaurant,Neighborhood,Music Venue,Music School,Museum,Multiplex,Accessories Store
7,0,Atfeh,Café,Egyptian Restaurant,Theater,Plaza,Coffee Shop,Falafel Restaurant,Pastry Shop,Movie Theater,...,Motorcycle Shop,Moving Target,Museum,New American Restaurant,Nightclub,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Indian Restaurant
8,0,Azbakeya,Café,Egyptian Restaurant,Theater,Plaza,Coffee Shop,Falafel Restaurant,Pastry Shop,Movie Theater,...,Motorcycle Shop,Moving Target,Museum,New American Restaurant,Nightclub,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Indian Restaurant
9,0,Bab El Sharia,Café,Historic Site,Plaza,Mosque,Sandwich Place,Market,Kebab Restaurant,Music Venue,...,Opera House,Nightclub,New American Restaurant,Neighborhood,Music School,Museum,Multiplex,Moving Target,Movie Theater,Indian Restaurant


In [427]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['region']=='El Wahat El Bahariya']

,Cluster Labels,region,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue


In [434]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

kl_clustering = kl_grouped.drop(["region"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

In [474]:

# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

kl_merged= neighbour
kl_merged = kl_merged.join(neighborhoods_venues_sorted.set_index('region'), on='region')
kl_merged.head()

,region,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,...,155th Most Common Venue,156th Most Common Venue,157th Most Common Venue,158th Most Common Venue,159th Most Common Venue,160th Most Common Venue,161th Most Common Venue,162th Most Common Venue,163th Most Common Venue,164th Most Common Venue
0,15 May City,"93,879",29.82983,31.36965,0.0,Coffee Shop,Café,Market,Bus Station,Pharmacy,...,Nightclub,Movie Theater,New American Restaurant,Neighborhood,Music Venue,Music School,Museum,Multiplex,Moving Target,Accessories Store
1,Abdeen,"40,450",30.04542,31.24210,0.0,Café,Egyptian Restaurant,Plaza,Theater,Coffee Shop,...,Movie Theater,Moving Target,Multiplex,Museum,Music School,New American Restaurant,Opera House,Optical Shop,Organic Grocery,Accessories Store
2,El Darb El Ahmar,"58,677",29.97769,31.26361,0.0,Burger Joint,Food Truck,Business Service,Café,Kebab Restaurant,...,Opera House,Moving Target,Nightclub,New American Restaurant,Neighborhood,Music Venue,Music School,Museum,Multiplex,Accessories Store
3,Ain Shams,"616,374",30.13542,31.32389,0.0,Gym,Seafood Restaurant,Egyptian Restaurant,Electronics Store,Café,...,Opera House,Moving Target,Nightclub,New American Restaurant,Neighborhood,Music Venue,Music School,Museum,Multiplex,Accessories Store
4,Amreya,"153,046",30.11238,31.29133,3.0,Bakery,Kebab Restaurant,Egyptian Restaurant,Café,Yoga Studio,...,Opera House,Moving Target,Nightclub,New American Restaurant,Neighborhood,Music Venue,Music School,Museum,Multiplex,Accessories Store


# Superimposing the Coffe shops Location on the Map in Cairo and Giza

In [588]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['region'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 1st Cluster

In [589]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,region,Coffee Shop,Cluster Labels,Population,Latitude,Longitude
59,Zamalek,0.100000,0,"14,993",30.05586,31.2239
11,Bulaq,0.114286,0,"48,301",30.04992,31.2486
11,Bulaq,0.114286,0,"963,201",30.04992,31.2486


# 2nd Cluster (The Best one for Cofee Shop Project)

In [590]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,region,Coffee Shop,Cluster Labels,Population,Latitude,Longitude
29,El Salam,0.000000,1,"154,267",30.157800,31.402070
30,El Sayeda Zeinab,0.014286,1,"136,722",30.032409,31.239037
32,El Sharabiya,0.000000,1,"187,806",30.070050,31.259640
33,El Shorouk,0.000000,1,"87,565",30.106880,31.288040
34,El Warraq,0.000000,1,"724,963",30.138270,31.386710
35,El Weili,0.014286,1,"79,548",30.072060,31.283490
36,El Zaher,0.000000,1,"72,101",30.062530,31.254790
37,El Zawya El Hamra,0.000000,1,"319,198",30.096000,31.266340
38,Giza,0.000000,1,"369,351",29.989460,31.216890
39,Hada'iq El Qobbah,0.000000,1,"317,092",30.088430,31.283510


# Third Cluster

In [591]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,region,Coffee Shop,Cluster Labels,Population,Latitude,Longitude
40,Heliopolis,0.042857,2,"134,549",30.091020,31.311370
25,El Nozha,0.028571,2,"231,987",30.111000,31.347490
3,Abdeen,0.057143,2,"40,450",30.045420,31.242100
31,El Segil,0.057143,2,"317,442",30.049920,31.248600
52,Ossim,0.057143,2,"384,527",30.049920,31.248600
49,New Cairo 2,0.057143,2,"90,960",30.049920,31.248600
8,Azbakeya,0.057143,2,"19,826",30.049920,31.248600
46,Nasr City 1,0.057143,2,"636,864",30.069660,31.344950
19,El Khalifa,0.028571,2,"105,578",29.964960,31.251520
27,El Saff,0.042857,2,"420,306",30.112211,31.614588
